# Transfer Learning Model 'DenseNet201' #

In [ ]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet201, preprocess_input


In [3]:
def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train')
     valid_path = os.path.join(base_dir, 'valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:\\Users\\s169753\\OneDrive - TU Eindhoven\\TUe\\Jaar 3\\Q3\\Project Imaging/train+val')
input = Input(input_shape)
# get the pretrained model, cut out the top layer
pretrained = DenseNet201(input_shape=input_shape, include_top=False, weights='imagenet')

#for layer in pretrained.layers[-4]:
#    layer.trainable = False

# Freeze the layers except the last 4 layers
#for i, layer in enumerate(pretrained.layers):
    #layer.trainable = False
#    print(layer.name)

Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.


In [4]:
for layer in pretrained.layers:
            layer.trainable = False
        
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)
    

for j in range(5,1,-1):
    string = 'conv'+str(j)
   # print(string)

    indices = [i for i, s in enumerate(layernames) if string in s]
        
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
    

    pretrained = DenseNet201(input_shape=input_shape, include_top=False, weights='imagenet')
    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.5)(output)
    output = Dense(1, activation='sigmoid')(output)

    # Check the trainable status of the individual layers
    #for layer in pretrained.layers:
    #    print(layer, layer.trainable)

    model = Model(input, output)

    # note the lower lr compared to the cnn example
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
    #
    # print a summary of the model on screen
    #model.summary()
        
    # save the model and weights
    model_name = 'Densenet weights ImageNet_2'+string
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'

    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)


    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]


    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20

    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training

    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)

    # print a summary of the model on screen
    #model.summary()

Epoch 1/10
225/225 [==============================] - 201s 895ms/step - loss: 0.4401 - acc: 0.8137 - val_loss: 0.3582 - val_acc: 0.8725

Epoch 00001: val_loss improved from inf to 0.35816, saving model to Densenet weights ImageNet_2conv5_weights.hdf5
Epoch 2/10
 32/225 [===>..........................] - ETA: 2:14 - loss: 0.3032 - acc: 0.8916

KeyboardInterrupt: 

In [ ]:
for layer in pretrained.layers:
            layer.trainable = False
        
layernames = []
for i,layer in enumerate(pretrained.layers):
    layernames.append(layer.name)
    

for j in range(5,0,-1):
    string = 'conv'+str(j)
   # print(string)

    indices = [i for i, s in enumerate(layernames) if string in s]
        
    for layer in pretrained.layers[min(indices):]:
        layer.trainable = True
    
    for i,layer in enumerate(pretrained.layers):
        print(string,layer.name,layer.trainable)
        

In [12]:
print(string)

conv1


In [13]:
for i,layer in enumerate(pretrained.layers):
    print(layer.name, layer.trainable)

input_8 False
zero_padding2d_11 False
conv1/conv True
conv1/bn True
conv1/relu True
zero_padding2d_12 True
pool1 True
conv2_block1_0_bn True
conv2_block1_0_relu True
conv2_block1_1_conv True
conv2_block1_1_bn True
conv2_block1_1_relu True
conv2_block1_2_conv True
conv2_block1_concat True
conv2_block2_0_bn True
conv2_block2_0_relu True
conv2_block2_1_conv True
conv2_block2_1_bn True
conv2_block2_1_relu True
conv2_block2_2_conv True
conv2_block2_concat True
conv2_block3_0_bn True
conv2_block3_0_relu True
conv2_block3_1_conv True
conv2_block3_1_bn True
conv2_block3_1_relu True
conv2_block3_2_conv True
conv2_block3_concat True
conv2_block4_0_bn True
conv2_block4_0_relu True
conv2_block4_1_conv True
conv2_block4_1_bn True
conv2_block4_1_relu True
conv2_block4_2_conv True
conv2_block4_concat True
conv2_block5_0_bn True
conv2_block5_0_relu True
conv2_block5_1_conv True
conv2_block5_1_bn True
conv2_block5_1_relu True
conv2_block5_2_conv True
conv2_block5_concat True
conv2_block6_0_bn True
conv

conv5_block6_2_conv True
conv5_block6_concat True
conv5_block7_0_bn True
conv5_block7_0_relu True
conv5_block7_1_conv True
conv5_block7_1_bn True
conv5_block7_1_relu True
conv5_block7_2_conv True
conv5_block7_concat True
conv5_block8_0_bn True
conv5_block8_0_relu True
conv5_block8_1_conv True
conv5_block8_1_bn True
conv5_block8_1_relu True
conv5_block8_2_conv True
conv5_block8_concat True
conv5_block9_0_bn True
conv5_block9_0_relu True
conv5_block9_1_conv True
conv5_block9_1_bn True
conv5_block9_1_relu True
conv5_block9_2_conv True
conv5_block9_concat True
conv5_block10_0_bn True
conv5_block10_0_relu True
conv5_block10_1_conv True
conv5_block10_1_bn True
conv5_block10_1_relu True
conv5_block10_2_conv True
conv5_block10_concat True
conv5_block11_0_bn True
conv5_block11_0_relu True
conv5_block11_1_conv True
conv5_block11_1_bn True
conv5_block11_1_relu True
conv5_block11_2_conv True
conv5_block11_concat True
conv5_block12_0_bn True
conv5_block12_0_relu True
conv5_block12_1_conv True
conv5

In [14]:
#for i,layer in enumerate(model.layers):
    #print(i,layer.name)

#for layer in model.layers:
#    print(layer.output_shape)
# get the data generators


# save the model and weights
model_name = 'Densenet weights ImageNet 5 off_2'
model_filepath = model_name + '.json'
weights_filepath = model_name + '_weights.hdf5'

model_json = model.to_json() # serialize model to JSON
with open(model_filepath, 'w') as json_file:
    json_file.write(model_json)


# define the model checkpoint and Tensorboard callbacks
checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
tensorboard = TensorBoard(os.path.join('logs', model_name))
callbacks_list = [checkpoint, tensorboard]


# train the model, note that we define "mini-epochs"
train_steps = train_gen.n//train_gen.batch_size//20
val_steps = val_gen.n//val_gen.batch_size//20

# since the model is trained for only 10 "mini-epochs", i.e. half of the data is
# not used during training

history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=10,
                    callbacks=callbacks_list)

# print a summary of the model on screen
#model.summary()

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,1600,3,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node densenet201_7/conv5_block23_0_bn/FusedBatchNorm}} = FusedBatchNorm[T=DT_FLOAT, _class=["loc:@train...chNormGrad"], data_format="NCHW", epsilon=1.001e-05, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](densenet201_7/conv5_block22_concat/concat, conv5_block23_0_bn_5/gamma/read, conv5_block23_0_bn_5/beta/read, densenet201_7/bn/Const, densenet201_7/bn/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_7/mul/_80427}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_50922_loss_7/mul", tensor_type=DT_FLOAT, _device="/job:loca